AOC [23.08](https://adventofcode.com/2023/day/8) Haunted Wasteland

First line of input: instructions on taking L/R option from list of directions

rest of input: directions = nodes with other nodes

Solution:
1. turn directions into a dictionary: {AA:(BB,CC)}
1. turn directions into a list of zeros and ones (L=0, R=1). 

Start at node AAA. Iterate through the directions. 
* Current node = node
* next node current_node[0] or current_node[1] depending on the directions. 



In [2]:
test_data1 = '''RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)'''

test_data2 = '''LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)'''

In [37]:
data = [x for x in test_data2.split('\n')]
data

['LLR', '', 'AAA = (BBB, BBB)', 'BBB = (AAA, ZZZ)', 'ZZZ = (ZZZ, ZZZ)']

In [38]:
instructions = data[0]
instructions = instructions.replace('L','0')
instructions = instructions.replace('R','1')

print(instructions)

import re
pattern = re.compile(r'[A-Z]{3}')

nodes_dd = {}
for line in data[2:]:
    head,_,tail = line.partition('=')
    nodes = re.findall(pattern,tail)
    nodes_dd[head.strip()] = (nodes[0],nodes[1])

nodes_dd

001


{'AAA': ('BBB', 'BBB'), 'BBB': ('AAA', 'ZZZ'), 'ZZZ': ('ZZZ', 'ZZZ')}

In [39]:
curr_node = 'AAA'
curr_inst = 0
while curr_node != 'ZZZ':
    next_node = nodes_dd[curr_node][int(instructions[curr_inst])]
    curr_inst += 1
    if curr_inst >= len(instructions):
        curr_inst = 0
    curr_node = next_node
    
    print(curr_inst,next_node,curr_node)


    


1 BBB BBB
2 AAA AAA
0 BBB BBB
1 AAA AAA
2 BBB BBB
0 ZZZ ZZZ


OK, my two test cases work.

Try the production data. . .

In [40]:
data = [x for x in test_data2.split('\n')]

instructions = data[0]
instructions = instructions.replace('L','0')
instructions = instructions.replace('R','1')

print(instructions)

import re
pattern = re.compile(r'[A-Z]{3}')

nodes_dd = {}
for line in data[2:]:
    head,_,tail = line.partition('=')
    nodes = re.findall(pattern,tail)
    nodes_dd[head.strip()] = (nodes[0],nodes[1])

curr_node = 'AAA'
curr_inst = 0
steps = 0
while curr_node != 'ZZZ':
    next_node = nodes_dd[curr_node][int(instructions[curr_inst])]
    curr_inst += 1
    steps += 1
    if curr_inst >= len(instructions):
        curr_inst = 0
    curr_node = next_node
    
    print(curr_inst,next_node,curr_node)

print(f"steps = {steps}")

001
1 BBB BBB
2 AAA AAA
0 BBB BBB
1 AAA AAA
2 BBB BBB
0 ZZZ ZZZ
steps = 6


In [2]:
#data = [x for x in test_data2.split('\n')]

with open ('2308input.txt') as f_in:
    data = [line for line in f_in.read().split('\n')]

instructions = data[0]
instructions = instructions.replace('L','0')
instructions = instructions.replace('R','1')

#print(instructions)

import re
pattern = re.compile(r'[A-Z]{3}')

nodes_dd = {}
for line in data[2:]:
    head,_,tail = line.partition('=')
    nodes = re.findall(pattern,tail)
    nodes_dd[head.strip()] = (nodes[0],nodes[1])

curr_node = 'AAA'
curr_inst = 0
steps = 0
while curr_node != 'ZZZ':
    next_node = nodes_dd[curr_node][int(instructions[curr_inst])]
    curr_inst += 1
    steps += 1
    if curr_inst >= len(instructions):
        curr_inst = 0
    curr_node = next_node
    
    #print(curr_inst,next_node,curr_node)

print(f"steps = {steps}")

steps = 12169


12169 is the correct answer.

Pretty amazing when my initial solution actually works out!

Part 2: Start at multiple starting points (nodes that end with "A") and continue until node that ends with Z.
* How many steps are needed to get to ALL ghosts on Z nodes at same time?

Solution:
1. Find the N starting nodes. (where the ghosts start following)
1. Run the instructions (nothing needs to be changed, except the `while` test) to find how many steps are needed for each ghost to get to Z.
1. Find the least-common-multiple of these values --> N steps for all cycles to coincide

I think this is so straight forward that I'm not going to mess around with test data! BOOM!

In [3]:
start_nodes = [k for k,v in nodes_dd.items() if k[2] == 'A']
start_nodes

['HVA', 'HHA', 'BVA', 'RSA', 'AAA', 'NPA']

In [4]:
start_nodes = [k for k,v in nodes_dd.items() if k[2] == 'A']

steps_list = []

for curr_node in start_nodes:
    print(curr_node)

    curr_inst = 0
    steps = 0
    while curr_node[2] != 'Z':
        next_node = nodes_dd[curr_node][int(instructions[curr_inst])]
        curr_inst += 1
        steps += 1
        if curr_inst >= len(instructions):
            curr_inst = 0
        curr_node = next_node

    print(f"steps to {curr_node} = {steps}")
    steps_list.append(steps)
    

HVA
steps to RFZ = 14999
HHA
steps to TNZ = 20093
BVA
steps to MLZ = 17263
RSA
steps to KPZ = 16697
AAA
steps to ZZZ = 12169
NPA
steps to BXZ = 20659


How to find the LCM (least common multiple) for six different numbers? [This site](https://www.calculatorsoup.com/calculators/math/lcm.php) describes how to do this with prime factorization.

1. Find all the prime factors of each given number.
1. List all the prime numbers found, as many times as they occur most often for any one given number.
1. Multiply the list of prime factors together to find the LCM.

I found a [prime_factors function](https://stackoverflow.com/questions/23287/algorithm-to-find-largest-prime-factor-of-a-number/412942#412942) that I could use.

In [63]:
def prime_factors(n):
    """Returns all the prime factors of a positive integer"""
    factors = []
    d = 2
    while n > 1:
        while n % d == 0:
            factors.append(d)
            n /= d
        d = d + 1

    return factors

for steps in steps_list:
    print(steps,prime_factors(steps))

14999 [53, 283]
20093 [71, 283]
17263 [61, 283]
16697 [59, 283]
12169 [43, 283]
20659 [73, 283]


Since all of these numbers are unique, we just need to multiply them all together!

In [73]:
primes = [] # set
for steps in steps_list:
    for x in prime_factors(steps):
        primes.append(x)

lcm = 1
for x in list(set(primes)):
    lcm *= x

print(f"{lcm:,}, {lcm}")


12,030,780,859,469, 12030780859469


Even for python on my fast(ish) computer, cycling through 12 TRILLION cycles would take a while....

In [75]:
cycles = 0
for x in steps_list:
    cycles += x

print(f"cycles = {cycles}")

cycles = 101880


Much faster just to cycle through 101,000 steps to find the six different paths the ghosts took. 